# Group 5
## Team members:
- Jianting Liu(8950907)
- David (8999846) 
- Marieth (9016702)


# Canadian Housing Market Analysis: 20-Year Review

## Use Case Summary & Hypothesis Testing

The analysis examines the relationship between housing prices, income, and population in Canada over a 20-year period. By incorporating population data alongside existing housing prices and income metrics, we can better understand the demographic pressures on housing affordability.

### Hypothesis

**Null Hypothesis (H0):**  
There is no significant correlation between population growth and housing price increases when controlling for income changes.

**Alternative Hypothesis (H1):**  
Population growth has a significant positive correlation with housing price increases, even after controlling for income changes.

This enhanced analysis will help determine whether population growth is a significant driver of housing prices beyond what can be explained by income changes alone.


Megeing data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

class IncomeDataProcessor:
    """
    A class to process income and house price data.
    """

    def __init__(self, income_data_path, original_data_path):
        """
        Initializes the class with the given data paths.

        Args:
            income_data_path (str): Path to the income data CSV file.
            original_data_path (str): Path to the original data Excel file.
        """
        self.income_data = pd.read_csv(income_data_path)
        self.original_data = pd.read_excel(original_data_path)

    def clean_data(self):
        """
        Cleans the income data by removing currency symbols and converting to numeric.
        """
        self.income_data['Median income'] = self.income_data['Median income'].str.replace('$', '').str.replace(',', '').astype(float)

    def convert_to_monthly(self):
        """
        Converts annual income to monthly income and creates a new DataFrame.
        """
        new_rows = []
        
        for index, row in self.income_data.iterrows():
            year = row["Reference year"]
            median_income = row["Median income"] / 12  # Convert to monthly income
            
            for month in range(1, 13):
                date_str = f"{year}-{month:02d}-01"
                date = pd.to_datetime(date_str, format='%Y-%m-%d')
                
                new_row = {
                    "Date": date,
                    "Median income": median_income,
                    "Count of families": row["Count of families"],
                    "Family type": row["Family type"],
                    "Geography": row["Geography"],
                    "Selected income concept": row["Selected income concept"]
                }
                new_rows.append(new_row)
        
        self.monthly_data = pd.DataFrame(new_rows)
        
        return self.monthly_data

    def merge_data(self):
        """
        Merges the income data with the original data based on the 'Date' column.
        """
        self.monthly_data['Date'] = pd.to_datetime(self.monthly_data['Date'], format='%Y-%m-%d')
        self.merged_data = pd.merge(self.original_data, self.monthly_data[['Date', 'Median income']], on='Date', how='left')
        # Filter data from 2005 to 2022
        self.merged_data = self.merged_data[(self.merged_data['Date'].dt.year >= 2005) & 
                                           (self.merged_data['Date'].dt.year <= 2022)]
        return self.merged_data
    
    def apply_kmeans_clustering(self, n_clusters=3):
        """
        Applies K-Means clustering on 'Median income' and 'Apartment_unit' columns.
        
        Args:
            n_clusters (int): Number of clusters to form.
        
        Returns:
            DataFrame with a new 'Cluster' column.
        """
        clustering_data = self.merged_data[['Median income', 'Apartment_unit']].dropna()
        
        # Data scaling
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(clustering_data)

        # K-Means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        clustering_data['Cluster'] = kmeans.fit_predict(scaled_data)
        
        # Plot clusters
        plt.figure(figsize=(10, 6))
        sns.scatterplot(
            x='Median income', y='Apartment_unit', 
            hue='Cluster', data=clustering_data, palette='viridis'
        )
        plt.title("Clustering of Income and Apartment Prices")
        plt.show()

        return clustering_data

    def save_data(self, output_path):
        """
        Saves the merged data to a CSV file.
        """
        self.merged_data.to_csv(output_path)
    
    def exploratory_data_analysis(self):
        """
        Performs exploratory data analysis, including:
            - Visualizing the relationship between income and house price
            - Histogram and box plot of income and apartment prices
        """

        # Histogramas de Median income y Apartment_unit
        plt.figure(figsize=(10, 10))

        # Histogram of Median income
        plt.subplot(2, 1, 1)
        sns.histplot(self.merged_data['Median income'].dropna(), kde=True)
        plt.title("Distribution of Median Income")

        # Histogram of Apartment_unit prices
        plt.subplot(2, 1, 2)
        sns.histplot(self.merged_data['Apartment_unit'].dropna(), kde=True)
        plt.title("Distribution of Apartment Prices")

        plt.tight_layout()
        plt.show()

        # Boxplots de Median income y Apartment_unit
        plt.figure(figsize=(10, 10))

        # Box plot of Median income
        plt.subplot(2, 1, 1)
        sns.boxplot(x=self.merged_data['Median income'].dropna())
        plt.title("Box Plot of Median Income")

        # Box plot of Apartment_unit prices
        plt.subplot(2, 1, 2)
        sns.boxplot(x=self.merged_data['Apartment_unit'].dropna())
        plt.title("Box Plot of Apartment Prices")

        plt.tight_layout()
        plt.show()

        # Scatter plot of income vs apartment prices
        sns.scatterplot(x='Median income', y='Apartment_unit', data=self.merged_data)
        plt.title("Income vs Apartment Prices")
        plt.show()


# Example usage
processor = IncomeDataProcessor('incomedata.csv', 'News_release_chart_data_August_2024.xlsx')
processor.clean_data()
processor.convert_to_monthly()
data = processor.merge_data()



We download the population data from the website and convert it to monthly data.
data source:https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=2005&cubeTimeFrame.endMonth=07&cubeTimeFrame.endYear=2024&referencePeriods=20050101%2C20240701

In [11]:
def convert_excel_format(input_file, output_file, merge_file):
    # Read Excel file
    df = pd.read_excel(input_file)
    
    # Get column names and data values
    dates = df.columns.tolist()[1:]  # Skip first column
    values = df.iloc[0, 1:].tolist()  # Skip first column
    
    # Create new DataFrame
    new_df = pd.DataFrame({
        'Date': dates,
        'Population': values
    })
    
    # Remove rows containing 'Date' or 'Population'
    new_df = new_df[~new_df['Date'].isin(['Date'])]
    
    # Convert quarterly data to date format
    new_df['Date'] = new_df['Date'].str.replace('Q', '').str.split().apply(
        lambda x: pd.to_datetime(f"{x[1]}-{int(x[0])*3-2}-01")
    )
    
    # Create empty lists for monthly data
    monthly_dates = []
    monthly_populations = []
    
    # Process each quarterly data
    for idx, row in new_df.iterrows():
        quarter_start = row['Date']
        population = row['Population']
        
        # Create data for three months in each quarter
        for month in range(3):
            monthly_date = quarter_start + pd.DateOffset(months=month)
            monthly_population = population / 3  # Distribute quarterly population evenly across months
            
            monthly_dates.append(monthly_date)
            monthly_populations.append(monthly_population)
    
    # Create new monthly DataFrame
    monthly_df = pd.DataFrame({
        'Date': monthly_dates,
        'Population': monthly_populations
    })
    
    # Sort by date
    monthly_df = monthly_df.sort_values('Date').reset_index(drop=True)
    
    # Format date column to 'YYYY-MM-DD'
    monthly_df['Date'] = monthly_df['Date'].dt.strftime('%Y-%m-%d')
    
    # Read CSV file to merge
    merge_df = pd.read_csv(merge_file)
    
    # Ensure Date columns have consistent format in both DataFrames
    merge_df['Date'] = pd.to_datetime(merge_df['Date']).dt.strftime('%Y-%m-%d')
    
    # Merge data
    final_df = pd.merge(merge_df, monthly_df, on='Date', how='left')
    
    # Save final result
    final_df.to_excel(output_file, index=False)
    
    return final_df

# Usage example
input_file = 'population.xlsx'
merge_file = 'mergeddata.csv'
output_file = 'final_merged_data.xlsx'

result = convert_excel_format(input_file, output_file, merge_file)
print(result.head())

         Date  Composite  One_storey  Two_storey  Townhouse  Apartment_unit  \
0  2005-01-01     239800      207700      302900     201700          172000   
1  2005-02-01     240500      208400      303300     202300          173000   
2  2005-03-01     241300      209200      304300     202900          173900   
3  2005-04-01     242000      210100      304800     203300          174600   
4  2005-05-01     242600      210600      305400     203700          175400   

   Median income    Population  
0         4487.5  1.069167e+07  
1         4487.5  1.069167e+07  
2         4487.5  1.069167e+07  
3         4487.5  1.071352e+07  
4         4487.5  1.071352e+07  
